In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput
from summer2 import Stratification
from scipy.integrate import quad

from copy import copy
import itertools

pd.options.plotting.backend = "plotly"


In [2]:
class Compartment:
    """
    A tuberculosis model compartment.
    """
    SUSCEPTIBLE = "susceptible"
    EARLY_LATENT = "early_latent"
    LATE_LATENT = "late_latent"
    INFECTIOUS = "infectious"
    ON_TREATMENT = "on_treatment"
    RECOVERED = "recovered"



#REPLICABLE_COMPARTMENTS = [Compartment.LATENT, Compartment.INFECTIOUS]

BASE_COMPARTMENTS = [
    Compartment.SUSCEPTIBLE,
    Compartment.EARLY_LATENT,
    Compartment.LATE_LATENT,
    Compartment.INFECTIOUS,
    Compartment.ON_TREATMENT,
    Compartment.RECOVERED,
]
INFECTIOUS_COMPS = [
    Compartment.INFECTIOUS,
    Compartment.ON_TREATMENT,
]

LATENT_COMPS = [
    Compartment.EARLY_LATENT,
    Compartment.LATE_LATENT,
]

In [3]:
model = CompartmentalModel(
    times=(1800, 2020),
    compartments=BASE_COMPARTMENTS,
    infectious_compartments=INFECTIOUS_COMPS,
    timestep=0.1,
)

In [4]:
model.set_initial_population(
    distribution=
    {
        Compartment.INFECTIOUS: 990.0, 
        Compartment.SUSCEPTIBLE: 10.0,
    },
)
strata = ['0', '5', '15', '35', '50']
def get_age_stratification() -> Stratification:
    # Create the stratification
    strat = Stratification(name="age", strata=strata, compartments=BASE_COMPARTMENTS)
    return strat

In [5]:
model.add_infection_frequency_flow(
    name="infection",
    contact_rate=1,
    source=Compartment.SUSCEPTIBLE,
    dest=Compartment.EARLY_LATENT,
)

model.add_infection_frequency_flow(
    "infection_from_latent",
    0.3,
    Compartment.LATE_LATENT,
    Compartment.EARLY_LATENT,
)
model.add_infection_frequency_flow(
    "infection_from_recovered",
    0.5,
    Compartment.RECOVERED,
    Compartment.EARLY_LATENT,
)

In [6]:
early_activation_dict = {
    0: 6.6e-3,
    5: 2.7e-3,
    15: 2.7e-4,
}
late_activation_dict = {
    0: 1.9e-11,
    5: 6.4e-6,
    15: 3.3e-6,   
}
stabilization_dict = {
    0: 1.2e-2,
    5: 1.2e-2,
    15: 5.4e-3,
}

In [7]:
def order_dict_by_keys(input_dict):
    """
    sort the input dictionary keys and return two separate lists with keys and values as lists with corresponding
        elements

    :param input_dict: dict
        dictionary to be sorted
    :return:
        :dict_keys: list
            sorted list of what were the dictionary keys
        : list
            values applicable to the sorted list of dictionary keys
    """
    dict_keys = list(input_dict.keys())
    dict_keys.sort()
    return dict_keys, [input_dict[key] for key in dict_keys]

In [8]:
def map_params(input_dict, modelled_age_groups):
    dict_keys, dict_values = order_dict_by_keys(input_dict)
    age_groups = modelled_age_groups
    params_vals = dict_values
    for age in modelled_age_groups:
        if age > dict_keys[-1]:
            #params.append(dict_values[-1])
            params_vals.append(dict_values[-1])
    
    return {str(key): value for key, value in zip(age_groups, params_vals)}

In [9]:
age_strat = get_age_stratification()

In [10]:
stabilisation_rate = 1
early_activation_rate = 1
late_activation_rate = 1
model.add_transition_flow(
    "stabilisation",
    stabilisation_rate,
    Compartment.EARLY_LATENT,
    Compartment.LATE_LATENT,
)
model.add_transition_flow(
    "early_activation",
    early_activation_rate,
    Compartment.EARLY_LATENT,
    Compartment.INFECTIOUS,
)
model.add_transition_flow(
    "late_activation",
    late_activation_rate,
    Compartment.LATE_LATENT,
    Compartment.INFECTIOUS,
    )
    # Add post-diseases flows
model.add_transition_flow(
    "self_recovery",
    0.21,
    Compartment.INFECTIOUS,
    Compartment.RECOVERED,
)

In [13]:
age_strat.set_flow_adjustments(
    "stabilisation", map_params(stabilization_dict, [int(i) for i in strata])
)
age_strat.set_flow_adjustments(
    "early_activation", map_params(early_activation_dict, [int(i) for i in strata])
)
age_strat.set_flow_adjustments(
    "late_activation", map_params(late_activation_dict, [int(i) for i in strata])
)


In [14]:
model.stratify_with(age_strat)

In [15]:
model.request_output_for_flow("progession", "stabilisation")

In [16]:
model.run()

In [17]:
outputs_df = model.get_outputs_df()
outputs_df.plot()